# Crop scenes output from ASF hyp3 for Mintpy

In [2]:
import os
import matplotlib
import numpy as np
import shapely
import geopandas as gpd
import rasterio as rio
import rasterio.mask 
import shutil

In [6]:
# load aoi
aoi = 'C:/Users/qbren/Desktop/taco/projects/atmospheric_correction/data_processing/shapefiles/asc_extent_sanjuans.shp'
aoi_gdf = gpd.read_file(aoi)
aoi_gdf_proj = aoi_gdf.to_crs('EPSG:32613')

In [7]:
# load, crop, and save rasters

main_dir = '/Users/qbren/Desktop/taco/projects/atmospheric_correction/data_processing/training_data'
raw_dir = 'asc_raw_sanjuans_noise'
crop_dir = 'asc_crop_sanjuans_noise'
igram_dirs = os.listdir(f'{main_dir}/{raw_dir}')

for idir in igram_dirs:
    cwd = f'{main_dir}/{raw_dir}/{idir}'
    os.chdir(cwd)
    allfiles = os.listdir(cwd)
    tif_list = []
    for fn in allfiles:
        if fn[-4:] == '.tif':
            tif_list.append(fn)
    os.mkdir(f'{main_dir}/{crop_dir}/{idir}')
    for tif in tif_list:
        tif_fn = f'{main_dir}/{raw_dir}/{idir}/{tif}'
        src = rio.open(tif_fn)
        print("Writing: ", tif)
        out_image, out_transform = rasterio.mask.mask(src, aoi_gdf_proj.geometry, crop=True)
        out_meta = src.meta
        out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})
        out_fn = f'{main_dir}/{crop_dir}/{idir}/{tif}'
        with rio.open(out_fn, "w", **out_meta) as dest:
            dest.write(out_image)
        src.close()
        
    for fn in allfiles:
        if fn[-4:] == '.txt' and fn[-7:] != '.md.txt':
            text_fn = fn
            shutil.copy(f'{main_dir}/{raw_dir}/{idir}/{text_fn}', f'{main_dir}/{crop_dir}/{idir}/{text_fn}')    

Writing:  S1BB_20170508T010853_20170520T010854_VVP012_INT40_G_ueF_8D29_amp.tif
Writing:  S1BB_20170508T010853_20170520T010854_VVP012_INT40_G_ueF_8D29_corr.tif
Writing:  S1BB_20170508T010853_20170520T010854_VVP012_INT40_G_ueF_8D29_dem.tif
Writing:  S1BB_20170508T010853_20170520T010854_VVP012_INT40_G_ueF_8D29_los_disp.tif
Writing:  S1BB_20170508T010853_20170520T010854_VVP012_INT40_G_ueF_8D29_unw_phase.tif
Writing:  S1BB_20170508T010853_20170520T010854_VVP012_INT40_G_ueF_8D29_vert_disp.tif
Writing:  S1BB_20170508T010853_20170520T010854_VVP012_INT40_G_ueF_8D29_water_mask.tif
Writing:  S1BB_20170520T010854_20170601T010854_VVP012_INT40_G_ueF_7F75_amp.tif
Writing:  S1BB_20170520T010854_20170601T010854_VVP012_INT40_G_ueF_7F75_corr.tif
Writing:  S1BB_20170520T010854_20170601T010854_VVP012_INT40_G_ueF_7F75_dem.tif
Writing:  S1BB_20170520T010854_20170601T010854_VVP012_INT40_G_ueF_7F75_los_disp.tif
Writing:  S1BB_20170520T010854_20170601T010854_VVP012_INT40_G_ueF_7F75_unw_phase.tif
Writing:  S1BB_

In [8]:
crop_dirs = os.listdir(f'{main_dir}/{crop_dir}')

for idir in crop_dirs:
    cwd = f'{main_dir}/{crop_dir}/{idir}'
    os.chdir(cwd)
    allfiles = os.listdir(cwd)
    unw_fn = ''
    for fn in allfiles:
        if fn[-9:] == 'phase.tif':
            unw_fn = fn
    unw_path = f'{main_dir}/{crop_dir}/{idir}/{unw_fn}'
    src = rio.open(unw_path)
    print(src.shape)
    src.close()   

(2531, 3886)
(2531, 3886)
(2531, 3886)
(2531, 3886)
(2531, 3886)
(2531, 3886)
(2531, 3886)
(2531, 3886)
(2531, 3886)
(2531, 3886)
(2531, 3886)
(2531, 3886)
(2531, 3886)
(2531, 3886)
(2531, 3886)
(2531, 3886)
(2531, 3886)
(2531, 3886)
(2531, 3886)
(2531, 3886)
(2531, 3886)
(2531, 3886)
(2531, 3886)
(2531, 3886)
(2531, 3886)
(2531, 3886)
(2531, 3886)
(2531, 3886)
(2531, 3886)
(2531, 3886)
(2531, 3886)
(2531, 3886)
(2531, 3886)
(2531, 3886)
(2531, 3886)
(2531, 3886)
(2531, 3886)
(2531, 3886)
(2531, 3886)
(2531, 3886)
(2531, 3886)
(2531, 3886)
(2531, 3886)
(2531, 3886)
(2531, 3886)
(2531, 3886)
(2531, 3886)
(2531, 3886)
(2531, 3886)
(2531, 3886)
(2531, 3886)
(2531, 3886)
(2531, 3886)
(2531, 3886)
(2531, 3886)
